In [1]:
import json

In [2]:
import re

def calculate_structure_score(data,flow_type):
    # Determine if we're evaluating 'product_defect' or 'storewide_query'
    flow_type = data.get("flow", "")
    
    if flow_type == "product_defect":
        # Product Defect Logic
        required_rows = [
            'customer_name', 'email', 'member_level', 'phone', 'username',
            'street_address', 'full_address', 'city', 'state', 'zip_code','num_products',
            'order_id', 'packaging', 'payment_method', 'products', 'purchase_date',
            'names', 'amounts'
        ]
        max_structure_score = 25  # 20 rows + 5 columns
    elif flow_type == "storewide_query":
        # Storewide Query Logic
        required_rows = [
            'customer_name', 'email', 'member_level', 'phone', 'username',
            'street_address', 'full_address', 'city', 'state', 'zip_code',
            'names', 'amounts'
        ]
        max_structure_score = 19  # 14 rows + 5 columns
    else:
        print("Unknown flow type")
        return 0

    structure_score = 0

    # Check for the presence of rows
    for row in required_rows:
        if any(row in section for section in data.values()):
            print(f"Found row: {row}")
            structure_score += 1
        else:
            print(f"Missing row: {row}")
    
    # Check for the presence of the 5 required columns
    required_columns = ['personal', 'order', 'product', 'flow', 'subflow']
    for column in required_columns:
        if column in data:
            structure_score += 1
        else:
            print(f"Missing column: {column}")

    # Check for flow and subflow separately if not already counted
    if 'flow' in data:
        structure_score += 1
    if 'subflow' in data:
        structure_score += 1
    
    print(f"Structure Score: {structure_score:.2f}")

    # Normalize structure score
    structure_score = (structure_score / max_structure_score) * 100
    print(f"Max Structure Score: {max_structure_score}")
    return structure_score 


def calculate_entity_score(data, flow_type):
    entity_score = 0
    max_scores = {'product_defect': 20, 'storewide_query': 14}

    personal = data.get('personal', {})
    if 'customer_name' in personal: 
        entity_score += 1
        print("Customer name is present and correct.")
    if 'email' in personal and '@' in personal['email']: 
        entity_score += 1
        print("Email is present and correctly formatted.")
    if 'member_level' in personal: 
        entity_score += 1
        print("Member level is present.")
    if 'phone' in personal and len(''.join(filter(str.isdigit, personal['phone']))) == 10: 
        entity_score += 1
        print("Phone number is present and correctly formatted.")
    if 'username' in personal: 
        entity_score += 1
        print("Username is present.")

    order = data.get('order', {})
    if 'street_address' in order and 'full_address' in order and order['street_address'] in order['full_address']: 
        entity_score += 1
        print("Street address matches full address.")
    if 'full_address' in order and 'zip_code' in order and order['zip_code'] in order['full_address']: 
        entity_score += 1
        print("ZIP code matches full address.")
    if 'city' in order and order['city'] in order['full_address']: 
        entity_score += 1
        print("City matches full address.")
    if 'num_products' in order: 
        entity_score += 1
        print("Number of products is present.")
    if 'order_id' in order and len(order['order_id']) == 10 and order['order_id'].isdigit(): 
        entity_score += 1
        print("Order ID is correctly formatted.")
    if 'packaging' in order: 
        entity_score += 1
        print("Packaging information is present.")
    if 'payment_method' in order: 
        entity_score += 1
        print("Payment method is present.")
    if 'products' in order: 
        entity_score += 1
        print("Products information is present.")
    if 'purchase_date' in order: 
        entity_score += 1
        print("Purchase date is present.")
    if 'state' in order and order['state'] in order['full_address']: 
        entity_score += 1
        print("State matches full address.")
    if 'zip_code' in order: 
        entity_score += 1
        print("ZIP code is present.")

    if flow_type == 'product_defect':
        product = data.get('product', {})
        if 'names' in product: 
            entity_score += 1
            print("Product names are present.")
        if 'amounts' in product: 
            entity_score += 1
            print("Product amounts are present.")
    elif flow_type == 'storewide_query':
        product = data.get('product', {})
        if 'names' in product and not product['names']: 
            entity_score += 1
            print("Product names are correctly empty for storewide query.")
        if 'amounts' in product and not product['amounts']: 
            entity_score += 1
            print("Product amounts are correctly empty for storewide query.")

    if data.get('flow') in ['product_defect', 'storewide_query']: 
        entity_score += 1
        print("Flow is correctly specified.")
    if data.get('subflow') in ['refund_status', 'timing_4', 'return_size']: 
        entity_score += 1
        print("Subflow is correctly specified.")

    entity_score = max(0, entity_score) / max_scores[flow_type] * 100

    return entity_score


def evaluate_data(data):
    flow_type = data['flow']
    
    structure_score = calculate_structure_score(data, flow_type)
    entity_score = calculate_entity_score(data, flow_type)

    total_score = (structure_score + entity_score) / 2

    return {
        'structure_score': structure_score,
        'entity_score': entity_score,
        'total_score': total_score
    }


# Example data
example_data_1 = {
  "personal": {
    "customer_name": "alessandro phoenix",
    "email": "aphoenix939@email.com",
    "member_level": "gold",
    "phone": "(727) 760-7806",
    "username": "aphoenix939"
  },
  "order": {
    "street_address": "8865 lexington ave",
    "full_address": "8865 lexington ave  la fayette, tx 86229",
    "city": "la fayette",
    "num_products": "1",
    "order_id": "7916676427",
    "packaging": "yes",
    "payment_method": "credit card",
    "products": "[{'brand': 'michael_kors', 'product_type': 'shirt', 'amount': 69, 'image_url': 'images/michael_kors-shirt.jpeg'}]",
    "purchase_date": "2019-11-20",
    "state": "tx",
    "zip_code": "86229"
  },
  "product": {
    "names": [
      "michael_kors shirt"
    ],
    "amounts": [
      69
    ]
  },
  "flow": "product_defect",
  "subflow": "refund_status"
}

example_data_2 = {
  "personal": {
    "customer_name": "joyce wu",
    "member_level": "bronze",
    "phone": "(859) 787-9085",
  },
  "order": {
    "street_address": "9998 lincoln ave",
    "full_address": "9998 lincoln ave  raleigh, mi 14573",
    "city": "raleigh",
    "state": "mi",
    "zip_code": "14573"
  },
  "product": {
    "names": [],
    "amounts": []
  },
  "flow": "storewide_query",
  "subflow": "timing_4"
}


In [3]:
# Evaluation
evaluation_1 = evaluate_data(example_data_1)
print('\n')
evaluation_2 = evaluate_data(example_data_2)

print(evaluation_1)
print('\n')
print(evaluation_2)

Found row: customer_name
Found row: email
Found row: member_level
Found row: phone
Found row: username
Found row: street_address
Found row: full_address
Found row: city
Found row: state
Found row: zip_code
Found row: num_products
Found row: order_id
Found row: packaging
Found row: payment_method
Found row: products
Found row: purchase_date
Found row: names
Found row: amounts
Structure Score: 25.00
Max Structure Score: 25
Customer name is present and correct.
Email is present and correctly formatted.
Member level is present.
Phone number is present and correctly formatted.
Username is present.
Street address matches full address.
ZIP code matches full address.
City matches full address.
Number of products is present.
Order ID is correctly formatted.
Packaging information is present.
Payment method is present.
Products information is present.
Purchase date is present.
State matches full address.
ZIP code is present.
Product names are present.
Product amounts are present.
Flow is correctl